In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, ElectraForSequenceClassification, AdamW
from tqdm.notebook import tqdm

In [ ]:
device = torch.device("cuda")

In [ ]:
class DiscordCmdDataset(Dataset):

  def __init__(self, csv_file):
    # 일부 값중에 NaN이 있음...
    self.dataset = pd.read_csv(csv_file, sep=',').dropna(axis=0)
    # 중복제거
    self.dataset.drop_duplicates(subset=['sentence'], inplace=True)
    self.tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-small-v3-discriminator")

    print(self.dataset.describe())

  def __len__(self):
    return len(self.dataset)

  def __getitem__(self, idx):
    row = self.dataset.iloc[idx, 0:2].values
    text = row[0]
    y = row[1] - 1

    inputs = self.tokenizer(
        text,
        return_tensors='pt',
        truncation=True,
        max_length=256,
        pad_to_max_length=True,
        add_special_tokens=True
        )

    input_ids = inputs['input_ids'][0]
    attention_mask = inputs['attention_mask'][0]

    return input_ids, attention_mask, y

In [ ]:
train_dataset = DiscordCmdDataset("train.csv")
test_dataset = DiscordCmdDataset("validation.csv")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/61.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/458 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/263k [00:00<?, ?B/s]

            label
count  272.000000
mean     2.268382
std      1.173835
min      1.000000
25%      1.000000
50%      2.000000
75%      3.000000
max      4.000000
           label
count  61.000000
mean    2.311475
std     1.148056
min     1.000000
25%     1.000000
50%     2.000000
75%     3.000000
max     4.000000


In [ ]:
model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-base-v3-discriminator", num_labels=4)
model.to(device)

#한번 실행해보기
text, attention_mask, y = train_dataset[0]
model(text.unsqueeze(0).to(device), attention_mask=attention_mask.unsqueeze(0).to(device))

config.json:   0%|          | 0.00/467 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/452M [00:00<?, ?B/s]

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2645: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


SequenceClassifierOutput(loss=None, logits=tensor([[-0.0474,  0.0252, -0.0032,  0.0123]], device='cuda:0',
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [ ]:
model

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(35000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): L

In [ ]:
model.load_state_dict(torch.load("/content/drive/MyDrive/model.pt"))

FileNotFoundError: [Errno 2] No such file or directory: 'model.pt'

In [ ]:
epochs = 5
batch_size = 16

In [ ]:
optimizer = AdamW(model.parameters(), lr=5e-6)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
import torch
import torch.nn.functional as F
from tqdm import tqdm

# Define loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()

# Training loop
losses = []
accuracies = []

for epoch in range(epochs):
    total_loss = 0.0
    correct = 0
    total = 0

    model.train()

    for input_ids_batch, attention_masks_batch, y_batch in tqdm(train_loader):
        optimizer.zero_grad()
        y_batch = y_batch.to(device)

        # Forward pass
        outputs = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))
        logits = outputs.logits

        # Calculate loss
        loss = criterion(logits, y_batch)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        # Calculate accuracy
        _, predicted = torch.max(logits, 1)
        correct += (predicted == y_batch).sum().item()
        total += y_batch.size(0)

    # Average loss and accuracy for the epoch
    epoch_loss = total_loss / len(train_loader)
    epoch_accuracy = correct / total

    losses.append(epoch_loss)
    accuracies.append(epoch_accuracy)

    print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}")

# Print training summary
print("Training finished.")


  0%|          | 0/17 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2645: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


tensor([[-0.0391,  0.0246, -0.0546,  0.0345],
        [-0.0454,  0.0261,  0.0170,  0.0186],
        [-0.0812, -0.0301, -0.0522,  0.0016],
        [-0.0497,  0.0537,  0.0065,  0.0678],
        [-0.0821, -0.0042,  0.0161, -0.0043],
        [-0.0290,  0.0656,  0.0636,  0.0932],
        [-0.1375,  0.0182, -0.0254,  0.0279],
        [-0.0942,  0.0538,  0.0434,  0.0416],
        [-0.0699,  0.0525, -0.0039,  0.0151],
        [-0.0263, -0.0220, -0.0752,  0.0691],
        [-0.1285,  0.0443,  0.0599,  0.0178],
        [-0.0385, -0.0264,  0.0099, -0.0130],
        [-0.0432,  0.0334,  0.0499,  0.0233],
        [-0.0943,  0.0844,  0.0480, -0.0080],
        [-0.0688, -0.0182,  0.0069,  0.0654],
        [-0.0404, -0.0313, -0.0583, -0.0050]], device='cuda:0',
       grad_fn=<AddmmBackward0>) tensor([2, 0, 2, 0, 0, 1, 1, 3, 2, 0, 1, 0, 1, 0, 0, 2], device='cuda:0')


  6%|▌         | 1/17 [00:01<00:17,  1.10s/it]

tensor([[-0.0718,  0.0108, -0.0030,  0.0340],
        [-0.0914,  0.0187,  0.0053,  0.0558],
        [-0.1045,  0.0137,  0.0769,  0.0239],
        [-0.1911, -0.0372, -0.0216,  0.0352],
        [-0.0862,  0.0125,  0.0799,  0.0556],
        [-0.0785,  0.0107,  0.0420, -0.0577],
        [-0.0813,  0.0572, -0.0137, -0.0126],
        [-0.0056, -0.0201,  0.0086, -0.0156],
        [-0.0335, -0.0182,  0.0118, -0.0115],
        [-0.0602,  0.0685, -0.0662,  0.0135],
        [ 0.0189,  0.0883,  0.0080, -0.0043],
        [-0.0032,  0.0096,  0.0083,  0.0526],
        [ 0.0052,  0.0652,  0.1038, -0.0155],
        [-0.1263,  0.0261,  0.0281,  0.0686],
        [-0.0731,  0.0348, -0.0048,  0.0515],
        [-0.0501, -0.0031,  0.0827,  0.0167]], device='cuda:0',
       grad_fn=<AddmmBackward0>) tensor([1, 0, 1, 1, 0, 2, 0, 0, 3, 2, 3, 0, 3, 3, 3, 1], device='cuda:0')


 12%|█▏        | 2/17 [00:01<00:12,  1.24it/s]

...

tensor([[ 0.0607, -0.2963, -0.1809,  0.3116],
        [ 0.1427,  0.0501, -0.2006, -0.0968],
        [ 0.0374,  0.2597, -0.1291, -0.2362],
        [ 0.1449,  0.0474, -0.0578, -0.1951],
        [ 0.1608,  0.1315, -0.1372, -0.2302],
        [ 0.1598, -0.3269, -0.1411,  0.2847],
        [ 0.5818, -0.2614, -0.2151, -0.1394],
        [ 0.0704, -0.1627, -0.2226,  0.2984],
        [ 0.0743,  0.1109, -0.0650, -0.1344],
        [ 0.5746, -0.2621, -0.2530, -0.2556],
        [ 0.1177,  0.0696,  0.0493, -0.2271],
        [ 0.1319, -0.0282, -0.0254, -0.1656],
        [-0.0429,  0.0588,  0.0239, -0.0713],
        [ 0.0553, -0.1780, -0.1887,  0.3193],
        [ 0.6523, -0.2622, -0.0979, -0.2378],
        [ 0.5566, -0.3404, -0.1850, -0.0811]], device='cuda:0',
       grad_fn=<AddmmBackward0>) tensor([3, 1, 1, 2, 1, 3, 0, 3, 2, 0, 2, 2, 2, 3, 0, 0], device='cuda:0')


 94%|█████████▍| 16/17 [00:10<00:00,  1.56it/s]

tensor([[ 0.4912, -0.1805, -0.2642, -0.1257],
        [ 0.0774,  0.1614,  0.0056, -0.1844],
        [-0.0459,  0.2667, -0.1976, -0.1853],
        [ 0.2679, -0.3303, -0.2262,  0.2706],
        [ 0.0027, -0.1971, -0.1950,  0.3897],
        [ 0.3869, -0.0951, -0.1647, -0.2090],
        [ 0.0839,  0.0135, -0.0328, -0.0580],
        [-0.1033, -0.0944, -0.1646,  0.2436],
        [-0.0382,  0.2006,  0.0058, -0.1268],
        [ 0.6542, -0.2217, -0.1825, -0.2173],
        [ 0.1105,  0.0523, -0.0213, -0.1347],
        [ 0.1271, -0.1245, -0.2609,  0.1422],
        [ 0.0310, -0.2059, -0.2279,  0.3160],
        [ 0.5999, -0.2987, -0.2027, -0.1376],
        [-0.0858, -0.2196, -0.2851,  0.3884],
        [ 0.5615, -0.2873, -0.1590, -0.1165]], device='cuda:0',
       grad_fn=<AddmmBackward0>) tensor([0, 2, 1, 3, 3, 0, 2, 3, 2, 0, 2, 3, 3, 0, 3, 0], device='cuda:0')


100%|██████████| 17/17 [00:10<00:00,  1.57it/s]

Epoch 5/5, Loss: 1.1244, Accuracy: 0.7243
Training finished.


In [ ]:
model.eval()

test_correct = 0
test_total = 0

for input_ids_batch, attention_masks_batch, y_batch in tqdm(test_loader):
  y_batch = y_batch.to(device)
  y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
  _, predicted = torch.max(y_pred, 1)
  test_correct += (predicted == y_batch).sum()
  test_total += len(y_batch)

print("Accuracy:", test_correct.float() / test_total)

100%|██████████| 4/4 [00:00<00:00,  5.44it/s]

Accuracy: tensor(0.8361, device='cuda:0')


In [ ]:
# 모델 저장하기
torch.save(model.state_dict(), "/content/drive/MyDrive/model.pt")